## Fundamentals
**SQLAlchemy ORM - Declaring Mapping**

The main goal of SQLAlchemy's Object Relational Mapper (ORM) is to link user-defined Python classes with database tables, synchronizing changes in object and row states.

**Declaration of Mapping:**

1. **Setting up Engine:** Use `create_engine()` to establish an engine object for database operations. It connects to the database and generates activity logs if `echo` is set to `True`. Example:
   ```python
   from sqlalchemy import create_engine
   engine = create_engine('db_url', echo=True)
   ```

2. **Configurational Process:** Begin by describing database tables and defining classes to map to these tables. This is done using SQLAlchemy's Declarative system, where classes include directives to describe the corresponding database tables.
   - Create a base class using `declarative_base()` from `sqlalchemy.ext.declarative`.
   - Define mapped classes with `__tablename__` attribute and Column definitions.
   - Each mapped class must have a primary key column.
   Example:
   ```python
   from sqlalchemy import Column, Integer, String
   from sqlalchemy.ext.declarative import declarative_base

   Base = declarative_base()

   class Customers(Base):
       __tablename__ = 'customers'
       id = Column(Integer, primary_key=True)
       name = Column(String)
       address = Column(String)
       email = Column(String)
   ```

3. **Table Metadata and Creation:** The Declarative system generates table metadata, represented by Table objects. These objects are associated with classes via a Mapper object. Use `Base.metadata.create_all(engine)` to create tables in the database based on the defined classes.
   Example:
   ```python
   Base.metadata.create_all(engine)
   ```

## Coding Examples

In [4]:
import random
from sqlalchemy import and_, or_, not_, text
from sqlalchemy.orm import sessionmaker
from models import User, engine

In [2]:
# Create session
Session = sessionmaker(bind=engine)
session = Session()

**CRUD Operations**

In [4]:
user1 = User(name="Amit", age=25)
user2 = User(name="Mobin", age=27)
user3 = User(name="Santu", age=28)
user4 = User(name="Morgan", age=40)

In [5]:
## CREATE 
session.add(user1)
session.add_all([user2, user3, user4])
session.commit()

In [6]:
## READ 
users = session.query(User).all()
for user in users:
    print(f"ID : {user.id}, Name : {user.name}, Age : {user.age}")

ID : 1, Name : Amit, Age : 25
ID : 2, Name : Mobin, Age : 27
ID : 3, Name : Santu, Age : 28
ID : 4, Name : Morgan, Age : 40


In [7]:
## UPDATE
user = session.query(User).filter_by(name = "Amit").one_or_none()
user.name = "Farooqe"
print(f"ID : {user.id}, Name : {user.name}, Age : {user.age}")

ID : 1, Name : Farooqe, Age : 25


In [8]:
## DELETE 
user = session.query(User).filter_by(id=4).one_or_none()
session.delete(user)
session.commit()

**Data Manipulation**

In [9]:
## Add some new data 

names = ["Thor", "Hulk", "Goku", "Vegeta", "Morgoth", "Frodo", "Pippin"]
ages = [20, 21, 22, 23, 24, 25, 26, 26, 27, 28, 29, 30]

for i in range(len(names)):
    user = User(name=names[i], age=random.choice(ages))
    session.add(user)
session.commit()

**Ordering data**

In [10]:
## SELECT * FROM users ORDER by AGE;
users = session.query(User).order_by(User.age).all()
for user in users:
    print(f"ID : {user.id}, Name : {user.name}, Age : {user.age}")

ID : 11, Name : Pippin, Age : 21
ID : 7, Name : Goku, Age : 22
ID : 10, Name : Frodo, Age : 22
ID : 9, Name : Morgoth, Age : 24
ID : 6, Name : Hulk, Age : 24
ID : 5, Name : Thor, Age : 24
ID : 1, Name : Farooqe, Age : 25
ID : 8, Name : Vegeta, Age : 26
ID : 2, Name : Mobin, Age : 27
ID : 3, Name : Santu, Age : 28


**Filtering Data**
- For general query statement with conditionals, use `filter()` or `where()` method.
- In order to find with specific value, use `filter_by()` method. This doesn't allow conditionals.

Normally inside the `filter()` and `where()` methods, the passed parameters are considered as **AND** operation by default. 

In [35]:
users = session.query(User).all() 
print(f"Total users : {len(users)}")

### SELECT * FROM users WHERE age>=25
filtered_users = session.query(User).filter(User.age >= 25).all() 
print(f"Filtered users Using single conditionals: {len(filtered_users)}")

### SELECT * FROM users WHERE age>=24 AND name="Thor"
filtered_users2 = session.query(User).filter(User.age>=24, User.name=="Thor").all()
print(f"Filtered users Using multiple conditionals: {len(filtered_users2)}")

filtered_users3 = session.query(User).filter_by(age=24, name="Thor").all()
print(f"Filtered user using specific values: {len(filtered_users3)}")

filtered_users4 = session.query(User).where(User.age>=24, User.name=="Thor").all()
print(f"Filtered users Using multiple conditionals: {len(filtered_users4)}")

Total users : 10
Filtered users Using single conditionals: 4
Filtered users Using multiple conditionals: 1
Filtered user using specific values: 1
Filtered users Using multiple conditionals: 1


1. In order to perform **AND** operation, there are two ways:
    - We can also use `and_` function.
    - We can use the **Bitwise** `&` operator. 

In [38]:
### SELECT * FROM users WHERE age>=23 OR name="Thor"

filtered_users5 = session.query(User).where(and_(User.age>=23, User.name=="Thor")).all()
print(f"Filtered users Using multiple conditionals (AND Operation): {len(filtered_users5)}")

filtered_users6 = session.query(User).where((User.age>=23) & (User.name=="Thor")).all()
print(f"Filtered users Using multiple conditionals (AND Operation): {len(filtered_users6)}")


Filtered users Using multiple conditionals (AND Operation): 1
Filtered users Using multiple conditionals (AND Operation): 1


2. In order to perform **OR** operation, there are two ways:
    - the passed parameters needs to be inside the `or_` function.
    - We can use **bitwise** `|` operator.

In [39]:
### SELECT * FROM users WHERE age>=23 OR name="Thor"

filtered_users3_1 = session.query(User).where(or_(User.age>=23, User.name=="Thor")).all()
print(f"Filtered users Using multiple conditionals (OR Operation): {len(filtered_users3_1)}")

filtered_users3_2 = session.query(User).where((User.age>=23) | (User.name=="Thor")).all()
print(f"Filtered users Using multiple conditionals (OR Operation): {len(filtered_users3_2)}")

Filtered users Using multiple conditionals (OR Operation): 7
Filtered users Using multiple conditionals (OR Operation): 7


3. In order to perform **NOT** operation, there are two ways:
    - the passed parameters needs to be inside the `not_` function.
    - We can use `!=` operator.

In [42]:
### SELECT * FROM users WHERE name!="Thor"

filtered_users7 = session.query(User).where(not_(User.name=="Thor")).all()
print(f"Filtered users Using multiple conditionals (NOT Operation): {len(filtered_users7)}")

filtered_users8 = session.query(User).where(User.name!="Thor").all()
print(f"Filtered users Using multiple conditionals (NOT Operation): {len(filtered_users8)}")

Filtered users Using multiple conditionals (NOT Operation): 9
Filtered users Using multiple conditionals (NOT Operation): 9
